<a href="https://colab.research.google.com/github/cfong32/key-sentence-extraction/blob/main/exp18_tfidf_sbert_gpt_analysis_chatgptDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 11.9 MB/s eta 0:00:00


In [3]:
# import packages

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datasets import load_dataset
from spacy.lang.en import English
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from rouge_score.rouge_scorer import RougeScorer
from itertools import cycle
from functools import partial
from textwrap import wrap
from IPython.display import HTML as html_print
pd.set_option('display.min_rows', 4)
tqdm.pandas()
tqdm = partial(tqdm, position=0, leave=True)
Ks = [1, 5, 10, 20, 40, 60, 80, 100]

## Download preprocessed GPT embeddings
Please refer to exp16 for detail

In [7]:
# download df_sent (GPT's sentence embeddings)
!wget -O exp16_df_sent_chatgptDS.dfpkl https://uoguelphca-my.sharepoint.com/:u:/g/personal/chungyan_uoguelph_ca/EU4S_My825pNmVwvS14uCTMBmVwq5hdd-qQlmL2YFIXbmA?download=1
df_sent = pd.read_pickle('exp16_df_sent_chatgptDS.dfpkl')
display(df_sent)

# save them into a dict
sent_emb_dict = df_sent.set_index('sentence').embedding.to_dict()

--2023-04-08 15:58:44--  https://uoguelphca-my.sharepoint.com/:u:/g/personal/chungyan_uoguelph_ca/EU4S_My825pNmVwvS14uCTMBmVwq5hdd-qQlmL2YFIXbmA?download=1
Resolving uoguelphca-my.sharepoint.com (uoguelphca-my.sharepoint.com)... 13.107.136.8, 13.107.138.8, 2620:1ec:8f8::8, ...
Connecting to uoguelphca-my.sharepoint.com (uoguelphca-my.sharepoint.com)|13.107.136.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/chungyan_uoguelph_ca/Documents/Shared/exp16_df_sent_chatgptDS.dfpkl?ga=1 [following]
--2023-04-08 15:58:45--  https://uoguelphca-my.sharepoint.com/personal/chungyan_uoguelph_ca/Documents/Shared/exp16_df_sent_chatgptDS.dfpkl?ga=1
Reusing existing connection to uoguelphca-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 57129307 (54M) [application/octet-stream]
Saving to: ‘exp16_df_sent_chatgptDS.dfpkl’

exp16_df_sent_chatg 100%[===================>]  54.48M  63.2MB/s    in 0.9s    

2023-04-08 15:58:46 (63.2 MB/s

,sentence,embedding
0,Climate change is one of the most pressing iss...,"[0.008533218875527382, -0.0168722253292799, 0...."
1,The rise in global temperatures due to human a...,"[-0.013889503665268421, -0.015929222106933594,..."
...,...,...
4591,"Conflicts and misunderstandings can arise, lea...","[-0.005323722027242184, -0.015705779194831848,..."
4592,"In such cases, it is important to communicate ...","[-0.00915580801665783, 0.009442524053156376, 0..."


In [8]:
# download df_art (GPT's article and hightlights embeddings)
!wget -O exp16_df_art_chatgptDS.dfpkl https://uoguelphca-my.sharepoint.com/:u:/g/personal/chungyan_uoguelph_ca/EaUsRfVNIl1HiGoLqKVvYD4Be0eeOIeFQET_W5mcmqBSyA?download=1
df_art = pd.read_pickle('exp16_df_art_chatgptDS.dfpkl')
display(df_art)

# save them into a dict
art_emb_dict  = df_art.set_index('article').article_emb.to_dict()

--2023-04-08 15:58:54--  https://uoguelphca-my.sharepoint.com/:u:/g/personal/chungyan_uoguelph_ca/EaUsRfVNIl1HiGoLqKVvYD4Be0eeOIeFQET_W5mcmqBSyA?download=1
Resolving uoguelphca-my.sharepoint.com (uoguelphca-my.sharepoint.com)... 13.107.136.8, 13.107.138.8, 2620:1ec:8f8::8, ...
Connecting to uoguelphca-my.sharepoint.com (uoguelphca-my.sharepoint.com)|13.107.136.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/chungyan_uoguelph_ca/Documents/Shared/exp16_df_art_chatgptDS.dfpkl?ga=1 [following]
--2023-04-08 15:58:54--  https://uoguelphca-my.sharepoint.com/personal/chungyan_uoguelph_ca/Documents/Shared/exp16_df_art_chatgptDS.dfpkl?ga=1
Reusing existing connection to uoguelphca-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 31849653 (30M) [application/octet-stream]
Saving to: ‘exp16_df_art_chatgptDS.dfpkl’

exp16_df_art_chatgp 100%[===================>]  30.37M  69.6MB/s    in 0.4s    

2023-04-08 15:58:55 (69.6 MB/s) -

,index,article,key_sentence,sentences,is_key,article_emb,key_sent_emb
0,0,Climate change is one of the most pressing iss...,We need to act urgently to reduce our greenhou...,[Climate change is one of the most pressing is...,"[0, 0, 0, 1]","[-0.0008796413894742727, -0.013974178582429886...","[-0.014789319597184658, -0.024009281769394875,..."
1,1,Education is the key to unlocking a better fut...,It is crucial that we invest in education at a...,[Education is the key to unlocking a better fu...,"[0, 0, 0, 1]","[-0.008852829225361347, -0.012599226087331772,...","[-0.006444304715842009, -0.015827901661396027,..."
...,...,...,...,...,...,...,...
1241,1258,The environment plays a critical role in our l...,It is our responsibility to take care of the e...,[The environment plays a critical role in our ...,"[0, 0, 0, 1]","[0.027159251272678375, -0.018947456032037735, ...","[0.0234543364495039, -0.018231311812996864, -0..."
1242,1259,Family is an essential component of our lives....,"In such cases, it is important to communicate ...",[Family is an essential component of our lives...,"[0, 0, 0, 0, 1]","[0.008778269402682781, 0.00871442724019289, 0....","[-0.009183181449770927, 0.009393483400344849, ..."


## Download preprocessed CNN/DM dataset + TFIDF + SBERT results
Please refer to exp6 for detail

In [9]:
# download df (TFIDF's results)
!wget -O exp17.dfpkl https://uoguelphca-my.sharepoint.com/:u:/g/personal/chungyan_uoguelph_ca/ERbg-h1wvFNLkfHqcSpWJHcB-D_mSlLPfDOXwtxra1H6Fw?download=1
df = (pd.read_pickle('exp17.dfpkl')
      .rename(columns={
          'TFIDF_sim'    : 'TFIDF_s2a_sim',        # sentence-to-article similarity
      })
     )
df

--2023-04-08 15:59:15--  https://uoguelphca-my.sharepoint.com/:u:/g/personal/chungyan_uoguelph_ca/ERbg-h1wvFNLkfHqcSpWJHcB-D_mSlLPfDOXwtxra1H6Fw?download=1
Resolving uoguelphca-my.sharepoint.com (uoguelphca-my.sharepoint.com)... 13.107.136.8, 13.107.138.8, 2620:1ec:8f8::8, ...
Connecting to uoguelphca-my.sharepoint.com (uoguelphca-my.sharepoint.com)|13.107.136.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/chungyan_uoguelph_ca/Documents/Shared/exp17.dfpkl?ga=1 [following]
--2023-04-08 15:59:16--  https://uoguelphca-my.sharepoint.com/personal/chungyan_uoguelph_ca/Documents/Shared/exp17.dfpkl?ga=1
Reusing existing connection to uoguelphca-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 1486323 (1.4M) [application/octet-stream]
Saving to: ‘exp17.dfpkl’

exp17.dfpkl         100%[===================>]   1.42M  --.-KB/s    in 0.02s   

2023-04-08 15:59:16 (57.6 MB/s) - ‘exp17.dfpkl’ saved [1486323/1486323]



,index,article,key_sentences,sentences,is_key,TFIDF_s2a_sim,pred_by_TFIDF,SBERT_s2a_sim,pred_by_SBERT
0,0,Climate change is one of the most pressing iss...,We need to act urgently to reduce our greenhou...,[Climate change is one of the most pressing is...,"[0, 0, 0, 1]","[0.3214220469953667, 0.6107510335598709, 0.474...","[0, 1, 0, 0]","[0.6913383, 0.67684746, 0.6024083, 0.76702094]","[0, 0, 0, 1]"
1,1,Education is the key to unlocking a better fut...,It is crucial that we invest in education at a...,[Education is the key to unlocking a better fu...,"[0, 0, 0, 1]","[0.47808946064127095, 0.45111889985171527, 0.4...","[0, 0, 0, 1]","[0.7876349, 0.5870048, 0.78181076, 0.79647756]","[0, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...
1241,1258,The environment plays a critical role in our l...,It is our responsibility to take care of the e...,[The environment plays a critical role in our ...,"[0, 0, 0, 1]","[0.4467579283397811, 0.6413572604770067, 0.556...","[0, 1, 0, 0]","[0.8151461, 0.4661282, 0.7739822, 0.8015023]","[1, 0, 0, 0]"
1242,1259,Family is an essential component of our lives....,"In such cases, it is important to communicate ...",[Family is an essential component of our lives...,"[0, 0, 0, 0, 1]","[0.4073544528009398, 0.42108696945190444, 0.57...","[0, 0, 1, 0, 0]","[0.75987756, 0.4105369, 0.7154193, 0.5800866, ...","[1, 0, 0, 0, 0]"


## Merge GPT results into `df`

In [11]:
# calculate sentence-to-article based on GPT embeddings
df['GPT_s2a_sim'] = df.progress_apply(
    lambda x: (
        cosine_similarity(
            np.stack([art_emb_dict[x.article]]),
            np.stack([sent_emb_dict[sent] for sent in x.sentences])
        )[0]
    ),
    axis=1
)

# predict by the one with the highest tfidf_cossim
# rank by sentence-to-article GPT embeddings
df['pred_by_GPT'] = df.GPT_s2a_sim.map(lambda x: (x==x.max()).astype(int))

df

100%|██████████| 1243/1243 [00:00<00:00, 2421.63it/s]


,index,article,key_sentences,sentences,is_key,TFIDF_s2a_sim,pred_by_TFIDF,SBERT_s2a_sim,pred_by_SBERT,GPT_s2a_sim,pred_by_GPT
0,0,Climate change is one of the most pressing iss...,We need to act urgently to reduce our greenhou...,[Climate change is one of the most pressing is...,"[0, 0, 0, 1]","[0.3214220469953667, 0.6107510335598709, 0.474...","[0, 1, 0, 0]","[0.6913383, 0.67684746, 0.6024083, 0.76702094]","[0, 0, 0, 1]","[0.9356893753165547, 0.9388009951553216, 0.856...","[0, 1, 0, 0]"
1,1,Education is the key to unlocking a better fut...,It is crucial that we invest in education at a...,[Education is the key to unlocking a better fu...,"[0, 0, 0, 1]","[0.47808946064127095, 0.45111889985171527, 0.4...","[0, 0, 0, 1]","[0.7876349, 0.5870048, 0.78181076, 0.79647756]","[0, 0, 0, 1]","[0.9343716371881718, 0.899088764749833, 0.9267...","[1, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
1241,1258,The environment plays a critical role in our l...,It is our responsibility to take care of the e...,[The environment plays a critical role in our ...,"[0, 0, 0, 1]","[0.4467579283397811, 0.6413572604770067, 0.556...","[0, 1, 0, 0]","[0.8151461, 0.4661282, 0.7739822, 0.8015023]","[1, 0, 0, 0]","[0.9218468893490073, 0.8721106516556567, 0.920...","[1, 0, 0, 0]"
1242,1259,Family is an essential component of our lives....,"In such cases, it is important to communicate ...",[Family is an essential component of our lives...,"[0, 0, 0, 0, 1]","[0.4073544528009398, 0.42108696945190444, 0.57...","[0, 0, 1, 0, 0]","[0.75987756, 0.4105369, 0.7154193, 0.5800866, ...","[1, 0, 0, 0, 0]","[0.9203702829455795, 0.8417133825817206, 0.909...","[1, 0, 0, 0, 0]"


# Analysis

In [13]:
for col in ['pred_by_TFIDF', 'pred_by_SBERT', 'pred_by_GPT']:
    accuracy = (df[col].map(np.argmax) == df.is_key.map(np.argmax)).mean()
    print(f'{col} accuracy: {accuracy:.2%}')

pred_by_TFIDF accuracy: 40.47%
pred_by_SBERT accuracy: 15.53%
pred_by_GPT accuracy: 6.52%


In [34]:
avg_rand_guess_accuray = np.mean([
    df.is_key.map(lambda x: x[np.random.randint(len(x))]).mean()
    for _ in range(999)
])
avg_rand_guess_accuray

0.28522730292641796

In [24]:
# save df
df.to_pickle('exp18.dfpkl')

In [26]:
# from google.colab import drive
# drive.mount('/gdrive')
# !cp exp18.dfpkl /gdrive/MyDrive/Shared/